### MyCanvas: Subklasse von Canvas mit neuen/modifizierten Methoden:
- `save_config()`
- `restore_config()`
- `draw_points(pts, point_size, color)`
- `draw_lines(pts, line_width, color)`
- `on_mouse_down(callback, remove = False)`  
   Wie vorher, aber ein bereits registrierter Callback wird &uuml;berschreiben.  
   Ebenso f&uuml;r die anderen Mouse-Events.

In [25]:
from ipycanvas import Canvas

# decorator
def preserve(f):
    def wrapper(self, *args):
        self.save_config()
        f(self, *args)
        self.restore_config()
    return wrapper    

# MyCanvas erbt von Canvas
# wird ein Attribut nicht in MyCanvas gefunden, so Canvas gesucht
# super() gibt ein (Proxy)-Objekt zurueck, mittels dem man auf
# Canvas zugreifen kann
class MyCanvas(Canvas):
    def __init__(self, width=200, height=200, **kwargs):
        self.defaults = {'fill_style': 'black', 'stroke_style': 'blue', 
                         'line_width': 2, 
                        }
        self.config = {}
        self.callbacks = {'on_mouse_down': None,
                          'on_mouse_move': None,
                          'on_mouse_up'  : None,
                         }
        self.draw = False
        
        # __init__ von Canvas aufrufen
        super().__init__(width=width, height=height)
        
        self.init(**kwargs)
        
    def init(self, **kwargs):
        for k in self.defaults:
            v = kwargs.get(k, self.defaults[k])
            setattr(self, k, v)
            
        self.draw_border()
    
    def save_config(self):
        for k in self.defaults:
            self.config[k] = getattr(self, k)
            
    def restore_config(self, config=None): 
        config = config or self.config
        for k,v in self.config.items():
            setattr(self, k, v)
      
    def _get(self, key):
        return self.kwargs.get(key, self.defaults.get(key))
        
    @preserve  
    def draw_border(self):
        self.line_width = 2
        self.stroke_color = 'blue'
        self.stroke_rect(0, 0, self.width, self.height)
        
    @preserve   
    def draw_points(self, pts, r, color):
        self.fill_style = color
        for pt in pts:
            self.fill_circle(*pt, r)   
            
    @preserve   
    def draw_lines(self, pts, lw, color):
        self.line_width = lw
        self.stroke_style = color
        for pt in pts:
            self.stroke_lines(pts)   
    
    def clear(self):
        # clear von Canvas aufrufen
        super().clear()
        self.draw_border()
        
    def on_mouse_down(self, f, remove = False):
        if remove:
            # on_mouse_down von Canvas aufrufen
            super().on_mouse_down(f_old, remove=True)
            self.callbacks['on_mouse_down'] = None
            return
        
        f_old = self.callbacks['on_mouse_down']
        if f_old:
            super().on_mouse_down(f_old, remove=True)
        self.callbacks['on_mouse_down'] = f
        super().on_mouse_down(f)
        
    def on_mouse_up(self, f, remove = False):
        if remove:
            super().on_mouse_up(f_old, remove=True)
            self.callbacks['on_mouse_up'] = None
            return
        
        f_old = self.callbacks['on_mouse_up']
        if f_old:
            super().on_mouse_up(f_old, remove=True)
        self.callbacks['on_mouse_up'] = f
        super().on_mouse_up(f)       
    
    def on_mouse_move(self, f, remove = False):
        if remove:
            super().on_mouse_move(f_old, remove=True)
            self.callbacks['on_mouse_move'] = None
            return
        
        f_old = self.callbacks['on_mouse_move']
        if f_old:
            super().on_mouse_move(f_old, remove=True)
        self.callbacks['on_mouse_move'] = f
        super().on_mouse_move(f)            

### Teste MyClass

In [26]:
canvas = MyCanvas(fill_style='red')
display(canvas)
canvas.fill_rect(10,10,50,50) # sollte rot sein

MyCanvas(height=200, width=200)

In [27]:
# border sollte bleiben
canvas.clear()

In [28]:
canvas.fill_circle(100,100,10)

In [29]:
import math
n = 10
r = 75
x,y = 100,100
tau = 2*math.pi
pts = [(r*math.cos(tau*i/n)+x, r*math.sin(tau*i/n)+y) for i in range(n)]
canvas.draw_lines(pts + [pts[0]], 1, 'orange')

In [30]:
pts = [(x,200-x) for x in range(0,201,10)]
canvas.draw_points(pts, 5, 'pink')

In [31]:
# Eventhandlers
def start_drawing(x,y):
    canvas.draw = True
    
def draw(x,y):
    if not canvas.draw:
        return
    canvas.fill_circle(x,y,5)

def stop_drawing(x,y):
    canvas.draw = False

In [32]:
# Register Eventhandlers
canvas.on_mouse_down(start_drawing)
canvas.on_mouse_up(stop_drawing)
canvas.on_mouse_move(draw)